In [6]:
def load_fdpde_output(fname, ft):

    import importlib
    import numpy as np

    
    # Load data - phase
    imod = importlib.import_module(fname+'_phase'+ft)
    data0 = imod._PETScBinaryLoad()

    # Get the phase value
    f0 = data0['X_cell']

    # Load data - solution
    fout = fname +  '_solution'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    xc = data['x1d_cell']
    zc = data['y1d_cell']
    xv = data['x1d_vertex']
    zv = data['y1d_vertex']
    vx = data['X_face_x']
    vz = data['X_face_y']
    p  = data['X_cell']

    fres0 = f0.reshape(mz,mx)

    # Prepare cell center velocities and pressure
    vxface = vx.reshape(mz  ,mx+1)
    vzface = vz.reshape(mz+1,mx  )
    pp = p.reshape(mz,mx)

    # Compute the cell center values from the face data by averaging neighbouring faces
    vxc = np.zeros( [mz , mx] )
    vzc = np.zeros( [mz , mx] )
    vc  = np.zeros( [mz , mx] )
    divv= np.zeros( [mz , mx] )
    dx  = np.zeros( [mx] )
    dz  = np.zeros( [mz] )
    dpdx = np.zeros( [mz , mx+1] )
    dpdz = np.zeros( [mz+1 , mx] )

    for i in range(0,mx):
        dx[i] = xv[i+1] - xv[i]
    for j in range(0,mz):
        dz[j] = zv[j+1] - zv[j]

    for i in range(0,mx):
        for j in range(0,mz):
            vxc[j][i] = 0.5 * (vxface[j][i+1] + vxface[j][i])
            vzc[j][i] = 0.5 * (vzface[j+1][i] + vzface[j][i])
            vc [j][i] = (vxc[j][i]**2+vzc[j][i]**2)**0.5
            divv[j][i] = (vxface[j][i+1] - vxface[j][i])/dx[i] + (vzface[j+1][i] - vzface[j][i])/dz[j]
            
            if i != 0:
                dpdx[j][i+1] =( pp[j][i] - pp[j][i-1])/dx[i]
            if j != 0:
                dpdz[j+1][i] = (pp[j][i] - pp[j-1][i])/dz[j]


    # Load data - coeff
    fout = fname+'_coefficient'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    etac_data = data['X_cell']
    etan = data['X_vertex']
    # split into dofs
    dof = 3
    etac = etac_data[1::dof]
    zetac = etac_data[2::dof] + 2/3*etac

    etacm = etac.reshape(mz,mx)
    zetacm = zetac.reshape(mz,mx)
    
    d2d3x = data['X_face_x']
    d2d3z = data['X_face_y']
    
    d2x = d2d3x[1::dof]
    d3x = d2d3x[2::dof]
    d2z = d2d3z[1::dof]
    d3z = d2d3z[2::dof]

    dd2x = d2x.reshape(mz,  mx+1)
    dd3x = d3x.reshape(mz,  mx+1)
    dd2z = d2z.reshape(mz+1,mx)
    dd3z = d3z.reshape(mz+1,mx)



    # Load data - hydrostatic pressure
    fout = fname+'_pstat'+ft  
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    ps = data['X_cell']
    pps = ps.reshape(mz,mx)

    #Load data - porosity
    fout = fname+'_phi'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    solid_phase = data['X_cell']
    dof = 1
    poro = 1- solid_phase[0::dof]
    poro_m = poro.reshape(mz,mx)

    #Load data - total deviatoric stress
    fout = fname+'_stress'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    s1n = data['X_vertex']
    s1c = data['X_cell']

    # split into dofs
    dof = 4
    sxx1c = s1c[0::dof]
    szz1c = s1c[1::dof]
    sxz1c = s1c[2::dof]
    sII1c = s1c[3::dof]

    sxx1n = s1n[0::dof]
    szz1n = s1n[1::dof]
    sxz1n = s1n[2::dof]
    sII1n = s1n[3::dof]

    sII1cm = sII1c.reshape(mz,mx)

    #Load data - DP
    fout = fname+'_dp'+ft # DP stress
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    dp_data = data['X_cell']
    dof = 4
    dp = dp_data[0::dof]

    dpm = dp.reshape(mz,mx)


    #Load data - plastic strain rates
    fout = fname+'_yield'+ft
    imod = importlib.import_module(fout)
    data = imod._PETScBinaryLoad()
    ys_data = data['X_cell']
    dof = 4
    ys = ys_data[0::dof]

    ysm = ys.reshape(mz,mx)

    # Load data - strain rates
    fout = fname+'_strain'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    eps1n = data['X_vertex']
    eps1c = data['X_cell']

    # split into dofs
    dof = 4
    exx1c = eps1c[0::dof]
    ezz1c = eps1c[1::dof]
    exz1c = eps1c[2::dof]
    eII1c = eps1c[3::dof]

    exx1n = eps1n[0::dof]
    ezz1n = eps1n[1::dof]
    exz1n = eps1n[2::dof]
    eII1n = eps1n[3::dof]

    # get the deviatoric strain rates
    divc = exx1c + ezz1c
    exx1c = exx1c - 1.0/3.0*divc
    ezz1c = ezz1c - 1.0/3.0*divc
    eII1c = np.power(0.5*(np.power(exx1c,2)+np.power(ezz1c,2)+2.0*np.power(exz1c,2)) ,0.5)

    divn = exx1n + ezz1n
    exx1n = exx1n - 1.0/3.0*divn
    ezz1n = ezz1n - 1.0/3.0*divn
    eII1n = np.power(0.5*(np.power(exx1n,2)+np.power(ezz1n,2)+2.0*np.power(exz1n,2)) ,0.5)

    eII1cm = eII1c.reshape(mz,mx)
    divcm = divc.reshape(mz,mx)
    
    
    #velociy of the fluid phase: vf = vs - 1/phi(d2*grad(p) - d3)
    vfx = np.zeros([mz, mx+1])
    vfz = np.zeros([mz+1, mx])
    vfxc = np.zeros([mz, mx])
    vfzc = np.zeros([mz, mx])


    phimin = 1e-4
  
    for i in range(0,mx+1):
        for j in range(0,mz+1):

            if (j!=mz):
                if i ==0:
                    phi = poro_m[j][i]
                elif i== mx:
                    phi = poro_m[j][mx-1]
                else:
                    phi = 0.5*(poro_m[j][i-1]+poro_m[j][i])
                if phi < phimin:
                    phi = phimin
                vfx[j][i] = vxface[j][i] + 1/phi *(dd2x[j][i] * dpdx[j][i] + dd3x[j][i])

            if (i!=mx):
                if j ==0:
                    phi = poro_m[j][i]
                elif j == mz:
                    phi = poro_m[j-1][i]
                else:
                    phi = 0.5*(poro_m[j-1][i]+poro_m[j][i])

                if phi< phimin:
                    phi = phimin

                vfz[j][i] = vzface[j][i] + 1/phi *(dd2z[j][i] * dpdz[j][i] + dd3z[j][i])

    for i in range(0,mx):
        for j in range(0,mz):
            vfxc[j][i] = 0.5 * (vfx[j][i+1] + vfx[j][i])
            vfzc[j][i] = 0.5 * (vfz[j+1][i] + vfz[j][i])

    
    return mx, mz, xc, zc, xv, zv, fres0, vxc, vzc, pp, pps, poro_m, eII1cm, divcm, sII1cm, dpm, ysm
#    return mx, mz, xc, zc, xv, zv, fres0, vxc, vzc, pp, pps, poro_m, eII1cm, divcm, sII1cm, dpm, ysm, vfxc, vfzc

In [9]:
def load_fdpde_output2(fname, ft):

    import importlib
    import numpy as np

    
    # Load data - phase
    imod = importlib.import_module(fname+'_phase'+ft)
    data0 = imod._PETScBinaryLoad()

    # Get the phase value
    f0 = data0['X_cell']

    # Load data - solution
    fout = fname +  '_solution'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    xc = data['x1d_cell']
    zc = data['y1d_cell']
    xv = data['x1d_vertex']
    zv = data['y1d_vertex']
    vx = data['X_face_x']
    vz = data['X_face_y']
    p  = data['X_cell']

    fres0 = f0.reshape(mz,mx)

    # Prepare cell center velocities and pressure
    vxface = vx.reshape(mz  ,mx+1)
    vzface = vz.reshape(mz+1,mx  )
    pp = p.reshape(mz,mx)

    # Compute the cell center values from the face data by averaging neighbouring faces
    vxc = np.zeros( [mz , mx] )
    vzc = np.zeros( [mz , mx] )
    vc  = np.zeros( [mz , mx] )
    divv= np.zeros( [mz , mx] )
    dx  = np.zeros( [mx] )
    dz  = np.zeros( [mz] )
    dpdx = np.zeros( [mz , mx+1] )
    dpdz = np.zeros( [mz+1 , mx] )

    for i in range(0,mx):
        dx[i] = xv[i+1] - xv[i]
    for j in range(0,mz):
        dz[j] = zv[j+1] - zv[j]

    for i in range(0,mx):
        for j in range(0,mz):
            vxc[j][i] = 0.5 * (vxface[j][i+1] + vxface[j][i])
            vzc[j][i] = 0.5 * (vzface[j+1][i] + vzface[j][i])
            vc [j][i] = (vxc[j][i]**2+vzc[j][i]**2)**0.5
            divv[j][i] = (vxface[j][i+1] - vxface[j][i])/dx[i] + (vzface[j+1][i] - vzface[j][i])/dz[j]
            
            if i != 0:
                dpdx[j][i+1] =( pp[j][i] - pp[j][i-1])/dx[i]
            if j != 0:
                dpdz[j+1][i] = (pp[j][i] - pp[j-1][i])/dz[j]


    # Load data - coeff
    fout = fname+'_coefficient'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    etac_data = data['X_cell']
    etan = data['X_vertex']
    # split into dofs
    dof = 3
    etac = etac_data[1::dof]
    zetac = etac_data[2::dof] + 2/3*etac

    etacm = etac.reshape(mz,mx)
    zetacm = zetac.reshape(mz,mx)
    
    d2d3x = data['X_face_x']
    d2d3z = data['X_face_y']
    
    d2x = d2d3x[1::dof]
    d3x = d2d3x[2::dof]
    d2z = d2d3z[1::dof]
    d3z = d2d3z[2::dof]

    dd2x = d2x.reshape(mz,  mx+1)
    dd3x = d3x.reshape(mz,  mx+1)
    dd2z = d2z.reshape(mz+1,mx)
    dd3z = d3z.reshape(mz+1,mx)



    # Load data - hydrostatic pressure
    fout = fname+'_pstat'+ft  
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    ps = data['X_cell']
    pps = ps.reshape(mz,mx)

    #Load data - porosity
    fout = fname+'_phi'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    solid_phase = data['X_cell']
    dof = 1
    poro = 1- solid_phase[0::dof]
    poro_m = poro.reshape(mz,mx)

    #Load data - total deviatoric stress
    fout = fname+'_stress'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    s1n = data['X_vertex']
    s1c = data['X_cell']

    # split into dofs
    dof = 4
    sxx1c = s1c[0::dof]
    szz1c = s1c[1::dof]
    sxz1c = s1c[2::dof]
    sII1c = s1c[3::dof]

    sxx1n = s1n[0::dof]
    szz1n = s1n[1::dof]
    sxz1n = s1n[2::dof]
    sII1n = s1n[3::dof]

    sII1cm = sII1c.reshape(mz,mx)

    #Load data - DP
    fout = fname+'_dp'+ft # DP stress
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    dp_data = data['X_cell']
    dof = 4
    dp = dp_data[0::dof]

    dpm = dp.reshape(mz,mx)


    #Load data - plastic strain rates
    fout = fname+'_yield'+ft
    imod = importlib.import_module(fout)
    data = imod._PETScBinaryLoad()
    ys_data = data['X_cell']
    dof = 4
    ys = ys_data[0::dof]

    ysm = ys.reshape(mz,mx)

    # Load data - strain rates
    fout = fname+'_strain'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    eps1n = data['X_vertex']
    eps1c = data['X_cell']

    # split into dofs
    dof = 4
    exx1c = eps1c[0::dof]
    ezz1c = eps1c[1::dof]
    exz1c = eps1c[2::dof]
    eII1c = eps1c[3::dof]

    exx1n = eps1n[0::dof]
    ezz1n = eps1n[1::dof]
    exz1n = eps1n[2::dof]
    eII1n = eps1n[3::dof]

    # get the deviatoric strain rates
    divc = exx1c + ezz1c
    exx1c = exx1c - 1.0/3.0*divc
    ezz1c = ezz1c - 1.0/3.0*divc
    eII1c = np.power(0.5*(np.power(exx1c,2)+np.power(ezz1c,2)+2.0*np.power(exz1c,2)) ,0.5)
    #reset the exx ezz value
    exx1c = exx1c + 1.0/3.0*divc
    ezz1c = ezz1c + 1.0/3.0*divc

    divn = exx1n + ezz1n
    exx1n = exx1n - 1.0/3.0*divn
    ezz1n = ezz1n - 1.0/3.0*divn
    eII1n = np.power(0.5*(np.power(exx1n,2)+np.power(ezz1n,2)+2.0*np.power(exz1n,2)) ,0.5)
    #reset the exx ezz value
    exx1n = exx1n + 1.0/3.0*divn
    ezz1n = ezz1n + 1.0/3.0*divn

    eII1cm = eII1c.reshape(mz,mx)
    divcm = divc.reshape(mz,mx)
    
    
    #velociy of the fluid phase: vf = vs - 1/phi(d2*grad(p) - d3)
    vfx = np.zeros([mz, mx+1])
    vfz = np.zeros([mz+1, mx])
    vfxc = np.zeros([mz, mx])
    vfzc = np.zeros([mz, mx])


    phimin = 1e-4
  
    for i in range(0,mx+1):
        for j in range(0,mz+1):

            if (j!=mz):
                if i ==0:
                    phi = poro_m[j][i]
                elif i== mx:
                    phi = poro_m[j][mx-1]
                else:
                    phi = 0.5*(poro_m[j][i-1]+poro_m[j][i])
                if phi < phimin:
                    phi = phimin
                vfx[j][i] = vxface[j][i] + 1/phi *(dd2x[j][i] * dpdx[j][i] + dd3x[j][i])

            if (i!=mx):
                if j ==0:
                    phi = poro_m[j][i]
                elif j == mz:
                    phi = poro_m[j-1][i]
                else:
                    phi = 0.5*(poro_m[j-1][i]+poro_m[j][i])

                if phi< phimin:
                    phi = phimin

                vfz[j][i] = vzface[j][i] + 1/phi *(dd2z[j][i] * dpdz[j][i] + dd3z[j][i])

    for i in range(0,mx):
        for j in range(0,mz):
            vfxc[j][i] = 0.5 * (vfx[j][i+1] + vfx[j][i])
            vfzc[j][i] = 0.5 * (vfz[j+1][i] + vfz[j][i])

    
    return exx1c, ezz1c, exz1c, sxx1c, szz1c, sxz1c

In [3]:
def load_fdpde_output_supp(fname, ft):

    import importlib
    import numpy as np

    
    # Load data - phase
    imod = importlib.import_module(fname+'_phase'+ft)
    data0 = imod._PETScBinaryLoad()

    # Get the phase value
    f0 = data0['X_cell']

    # Load data - solution
    fout = fname +  '_solution'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    xc = data['x1d_cell']
    zc = data['y1d_cell']
    xv = data['x1d_vertex']
    zv = data['y1d_vertex']
    vx = data['X_face_x']
    vz = data['X_face_y']
    p  = data['X_cell']

    fres0 = f0.reshape(mz,mx)

    # Prepare cell center velocities and pressure
    vxface = vx.reshape(mz  ,mx+1)
    vzface = vz.reshape(mz+1,mx  )
    pp = p.reshape(mz,mx)

    # prepare dataset to compute dpdx and dpdz
    dx  = np.zeros( [mx] )
    dz  = np.zeros( [mz] )
    dpdx = np.zeros( [mz , mx+1] )
    dpdz = np.zeros( [mz+1 , mx] )

    for i in range(0,mx):
        dx[i] = xv[i+1] - xv[i]
    for j in range(0,mz):
        dz[j] = zv[j+1] - zv[j]

    for i in range(0,mx):
        for j in range(0,mz):

            if i != 0:
                dpdx[j][i+1] =( pp[j][i] - pp[j][i-1])/dx[i]
            if j != 0:
                dpdz[j+1][i] = (pp[j][i] - pp[j-1][i])/dz[j]


    # Load data - coeff
    fout = fname+'_coefficient'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    dof = 3
    
    d2d3x = data['X_face_x']
    d2d3z = data['X_face_y']
    
    d2x = d2d3x[1::dof]
    d3x = d2d3x[2::dof]
    d2z = d2d3z[1::dof]
    d3z = d2d3z[2::dof]

    dd2x = d2x.reshape(mz,  mx+1)
    dd3x = d3x.reshape(mz,  mx+1)
    dd2z = d2z.reshape(mz+1,mx)
    dd3z = d3z.reshape(mz+1,mx)


    #Load data - total deviatoric stress
    fout = fname+'_stressold'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    s1old = data['X_cell']

    # split into dofs
    dof = 4
    sxx1co = s1old[0::dof]
    szz1co = s1old[1::dof]
    sxz1co = s1old[2::dof]
    sII1co = s1old[3::dof]

    sxxo = sxx1co.reshape(mz,mx)
    szzo = szz1co.reshape(mz,mx)
    sxzo = sxz1co.reshape(mz,mx)

    #Load data - DPold
    fout = fname+'_dpold'+ft # DP stress
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    dp_data = data['X_cell']
    dof = 4
    dpold = dp_data[0::dof]

    dpo = dpold.reshape(mz,mx)

    
    return xc, zc, fres0, vxface, vzface, dd2x, dd2z, dd3x, dd3z, sxxo, szzo, sxzo, dpo


In [1]:
def dyke_width_height(nx,n,xx,zz,dt,itmax,phi,phicut):

    import numpy as np

    dw = np.zeros([itmax,n,2]) #width
    dh = np.zeros([itmax,2])

    for i in range(itmax):

        #load time steps
        dw[i,:,0] = dt * i
        dh[i,0] = dt * i

        #height
        s = np.flatnonzero(phi[:,int(nx/2),i] > phicut)
        if (np.size(s) > 0):
            itip = s[-1]
        else:
            itip = 0
        dh[i,1] = zz[itip]

        #width
        for j in range(n):
            s = np.flatnonzero(phi[j,:,i] > phicut)
            if (np.size(s)>0):
                imin = s[0]
                imax = s[-1]
                dw[i,j,1] = xx[imax] - xx[imin]
            else:
                dw[i,j,1] = 0
                
    return dw, dh

In [3]:
def load_rift(nx,n,dt,itmax, ls, es, taus, c, ct, theta):

    phi = np.zeros([n,nx,itmax])
    dlam = np.zeros([n,nx,itmax])
    eii = np.zeros([n,nx,itmax])
    sii = np.zeros([n,nx,itmax])
    dp = np.zeros([n,nx,itmax])
    div = np.zeros([n,nx,itmax])
    pf = np.zeros([n,nx,itmax])
    vvxc = np.zeros([n,nx,itmax])
    vvzc = np.zeros([n,nx,itmax])
    eevp = np.zeros([n,nx,itmax])

    eexx = np.zeros([n,nx,itmax])
    eezz = np.zeros([n,nx,itmax])
    eexz = np.zeros([n,nx,itmax])
    ssxx = np.zeros([n,nx,itmax])
    sszz = np.zeros([n,nx,itmax])
    ssxz = np.zeros([n,nx,itmax])

    for i in range(itmax):

      #  print('i=', i)
        t = i
        if (i<=9):
            ft = '_ts00'+str(t)
        elif (i<=99):
            ft = '_ts0'+str(t)
        else:
            ft = '_ts' + str(t)

        fname = fname0 + '_n'+str(n)
        mx1, mz1, xc1, zc1, xv1, zv1, fres01, vxc1, vzc1, pp1, pps1,poro_m1, eII1cm1, divcm1, sII1cm1, dpm1, ysm1 = load_fdpde_output(fname, ft)

        exx, ezz, exz, sxx, szz, sxz = load_fdpde_output2(fname, ft)
        exxm1 = exx.reshape(mz1, mx1)
        ezzm1 = ezz.reshape(mz1, mx1)
        exzm1 = exz.reshape(mz1, mx1)
        sxxm1 = sxx.reshape(mz1, mx1)
        szzm1 = szz.reshape(mz1, mx1)
        sxzm1 = sxz.reshape(mz1, mx1)


        xc = xc1 *ls
        zc = zc1 *ls
        xv = xv1 *ls *es
        zv = zv1 *ls *es
        fres0 = fres01
        vxc = vxc1 *ls *es
        vzc = vzc1 *ls *es
        pp = pp1 * taus
        pps = pps1 * taus
        poro_m = poro_m1 
        eII1cm = eII1cm1 *es
        divcm = divcm1 *es
        sII1cm = sII1cm1 *taus
        dpm = dpm1 *taus
        ysm = ysm1 *es

        exxm = exxm1 * es
        ezzm = ezzm1 * es
        exzm = exzm1 * es
        sxxm = sxxm1 * taus
        szzm = szzm1 * taus
        sxzm = sxzm1 * taus

        sint = np.sin(theta)
        cost = np.cos(theta)

        evp1 = ysm1 * sII1cm1/(sII1cm1**2 + (c*cost-ct*sint)**2)**0.5
        evp = evp1 * es
        for jj in range(mz1):
            for ii in range(mx1):
                if (fres0[jj,ii] < 0.5 and zc1[jj] > 0.5 ):
                    poro_m[jj,ii] = np.nan
                    vxc[jj,ii] = np.nan
                    vzc[jj,ii] = np.nan
                    eII1cm[jj,ii] = np.nan
                    divcm[jj,ii] = np.nan
                    sII1cm[jj,ii] = np.nan
                    dpm[jj,ii] = np.nan
                    ysm[jj,ii] = np.nan
                    evp[jj,ii] = np.nan
                    pp[jj,ii] = np.nan
                    exxm[jj,ii] = np.nan
                    ezzm[jj,ii] = np.nan
                    exzm[jj,ii] = np.nan
                    sxxm[jj,ii] = np.nan
                    szzm[jj,ii] = np.nan
                    sxzm[jj,ii] = np.nan

        xc2 = xc
        phi[:,:,i] = poro_m
        dlam[:,:,i] = ysm
        eii[:,:,i] = eII1cm
        sii[:,:,i] = sII1cm
        dp[:,:,i] = dpm
        div[:,:,i] = divcm
        pf[:,:,i] = pp
        vvxc[:,:,i] = vxc
        vvzc[:,:,i] = vzc
        eevp[:,:,i] = evp

        eexx[:,:,i] = exxm
        eezz[:,:,i] = ezzm
        eexz[:,:,i] = exzm
        ssxx[:,:,i] = sxxm
        sszz[:,:,i] = szzm
        ssxz[:,:,i] = sxzm
    return xc, zc, phi, dlam, eii, sii, dp, div, pf, vvxc, vvzc, eevp, eexx, eezz, eexz, ssxx, sszz, ssxz

In [1]:
def load_rift2(nx,n,dt,itmax, ls, es, taus, c, ct, theta):

    
    phase = np.zeros([n,nx,itmax])
    
    phi = np.zeros([n,nx,itmax])
    dlam = np.zeros([n,nx,itmax])
    eii = np.zeros([n,nx,itmax])
    sii = np.zeros([n,nx,itmax])
    dp = np.zeros([n,nx,itmax])
    div = np.zeros([n,nx,itmax])
    pf = np.zeros([n,nx,itmax])
    vvxc = np.zeros([n,nx,itmax])
    vvzc = np.zeros([n,nx,itmax])
    eevp = np.zeros([n,nx,itmax])

    eexx = np.zeros([n,nx,itmax])
    eezz = np.zeros([n,nx,itmax])
    eexz = np.zeros([n,nx,itmax])
    ssxx = np.zeros([n,nx,itmax])
    sszz = np.zeros([n,nx,itmax])
    ssxz = np.zeros([n,nx,itmax])

    for i in range(itmax):

        t = i
        if (i<=9):
            ft = '_ts00'+str(t)
        elif (i<=99):
            ft = '_ts0'+str(t)
        else:
            ft = '_ts' + str(t)

        fname = fname0 + '_n'+str(n)
        mx1, mz1, xc1, zc1, xv1, zv1, fres01, vxc1, vzc1, pp1, pps1,poro_m1, eII1cm1, divcm1, sII1cm1, dpm1, ysm1 = load_fdpde_output(fname, ft)

    
        fres0 = fres01
   
    
        phase[:,:,i] = fres0
        
        print(i)
    return phase

In [2]:
def load_rift3(nx,n,dt,itmax, ls, es, taus, c, ct, theta):

    
    vxf = np.zeros([n,  nx+1, itmax])
    vzf = np.zeros([n+1,nx,   itmax])
    dd2x = np.zeros([n,  nx+1, itmax])
    dd2z = np.zeros([n+1,nx,   itmax])
    dd3x = np.zeros([n,  nx+1, itmax])
    dd3z = np.zeros([n+1,nx,   itmax])
    sxxo = np.zeros([n,nx,itmax])
    szzo = np.zeros([n,nx,itmax])
    sxzo = np.zeros([n,nx,itmax])
    dpo = np.zeros([n,nx,itmax])
    

    for i in range(itmax):

      #  print('i=', i)
        t = i
        if (i<=9):
            ft = '_ts00'+str(t)
        elif (i<=99):
            ft = '_ts0'+str(t)
        else:
            ft = '_ts' + str(t)

        fname = fname0 + '_n'+str(n)

        xc1, zc1, fres01, vxf1, vzf1, dd2x1, dd2z1, dd3x1, dd3z1, sxxo1, szzo1, sxzo1, dpo1 = load_fdpde_output_supp(fname, ft)
        

        xc = xc1 *ls
        zc = zc1 *ls
        fres0 = fres01
        
        vxf2 = vxf1 *ls *es
        vzf2 = vzf1 *ls *es
        
        dd2x2 = dd2x1 *ls*ls*es/taus
        dd2z2 = dd2z1 *ls*ls*es/taus
        dd3x2 = dd3x1 *ls*es
        dd3z2 = dd3z1 *ls*es
        
        sxxo2 = sxxo1 * taus
        szzo2 = szzo1 * taus
        sxzo2 = sxzo1 * taus
        
        dpo2 = dpo1 * taus
        
        
        for jj in range(n):
            for ii in range(nx):
                if (fres0[jj,ii] < 0.5 and zc1[jj] > 0.5 ):
                    sxxo2[jj,ii] = np.nan
                    szzo2[jj,ii] = np.nan
                    sxzo2[jj,ii] = np.nan
                    dpo2[jj,ii] = np.nan
                    
                    vxf2[jj,ii+1] = np.nan
                    dd2x2[jj, ii+1] = np.nan
                    dd3x2[jj, ii+1] = np.nan
                    
                    vzf2[jj+1, ii] = np.nan
                    dd2z2[jj+1, ii] =np.nan
                    dd3z2[jj+1, ii] = np.nan
                    
                    if (ii==0):
                        vxf2[jj,  ii] = np.nan
                        dd2x2[jj, ii] = np.nan
                        dd3x2[jj, ii] = np.nan
                    
                    if (jj==0):
                        vzf2[jj, ii] = np.nan
                        dd2z2[jj, ii] =np.nan
                        dd3z2[jj, ii] = np.nan
                    

        vxf[:,:,i] = vxf2
        vzf[:,:,i] = vzf2
        
        dd2x[:,:,i] = dd2x2
        dd2z[:,:,i] = dd2z2
        dd3x[:,:,i] = dd3x2
        dd3z[:,:,i] = dd3z2
        
        sxxo[:,:,i] = sxxo2
        szzo[:,:,i] = szzo2
        sxzo[:,:,i] = sxzo2
        
        dpo[:,:,i] = dpo2
        
    return vxf, vzf, dd2x, dd2z, dd3x, dd3z, sxxo, szzo, sxzo, dpo

In [1]:
def load_fdpde_coeff(fname, ft):

    import importlib
    import numpy as np

    
    # Load data - coeff
    fout = fname+'_coefficient'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    
    etac_data = data['X_cell']
    etan = data['X_vertex']
    # split into dofs
    dof = 3
    etac = etac_data[1::dof]
    zetac = etac_data[2::dof] + 2/3*etac

    etacm = etac.reshape(mz,mx)
    zetacm = zetac.reshape(mz,mx)
    
    d2d3x = data['X_face_x']
    d2d3z = data['X_face_y']
    
    d2x = d2d3x[1::dof]
    d3x = d2d3x[2::dof]
    d2z = d2d3z[1::dof]
    d3z = d2d3z[2::dof]

    dd2x = d2x.reshape(mz,  mx+1)
    dd3x = d3x.reshape(mz,  mx+1)
    dd2z = d2z.reshape(mz+1,mx)
    dd3z = d3z.reshape(mz+1,mx)


    
    return etacm, zetacm, dd2x, dd3x, dd2z, dd3z


In [1]:
def load_fdpde_phibk(fname, ft):

    import importlib
    import numpy as np

    
    # Load data - phase
    imod = importlib.import_module(fname+'_phase'+ft)
    data0 = imod._PETScBinaryLoad()
    
    # Get the phase value
    f0 = data0['X_cell']

    # Load data - solution
    fout = fname +  '_solution'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    xc = data['x1d_cell']
    zc = data['y1d_cell']
    xv = data['x1d_vertex']
    zv = data['y1d_vertex']
    vx = data['X_face_x']
    vz = data['X_face_y']
    p  = data['X_cell']

    fres0 = f0.reshape(mz,mx)

    # Prepare cell center velocities and pressure
    vxface = vx.reshape(mz  ,mx+1)
    vzface = vz.reshape(mz+1,mx  )
    pp = p.reshape(mz,mx)

    # Compute the cell center values from the face data by averaging neighbouring faces
    vxc = np.zeros( [mz , mx] )
    vzc = np.zeros( [mz , mx] )
    vc  = np.zeros( [mz , mx] )
    divv= np.zeros( [mz , mx] )
    dx  = np.zeros( [mx] )
    dz  = np.zeros( [mz] )
    dpdx = np.zeros( [mz , mx+1] )
    dpdz = np.zeros( [mz+1 , mx] )

    for i in range(0,mx):
        dx[i] = xv[i+1] - xv[i]
    for j in range(0,mz):
        dz[j] = zv[j+1] - zv[j]

    for i in range(0,mx):
        for j in range(0,mz):
            vxc[j][i] = 0.5 * (vxface[j][i+1] + vxface[j][i])
            vzc[j][i] = 0.5 * (vzface[j+1][i] + vzface[j][i])
            vc [j][i] = (vxc[j][i]**2+vzc[j][i]**2)**0.5
            divv[j][i] = (vxface[j][i+1] - vxface[j][i])/dx[i] + (vzface[j+1][i] - vzface[j][i])/dz[j]
            
            if i != 0:
                dpdx[j][i+1] =( pp[j][i] - pp[j][i-1])/dx[i]
            if j != 0:
                dpdz[j+1][i] = (pp[j][i] - pp[j-1][i])/dz[j]


    # Load data - coeff
    fout = fname+'_coefficient'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    etac_data = data['X_cell']
    etan = data['X_vertex']
    # split into dofs
    dof = 3
    etac = etac_data[1::dof]
    zetac = etac_data[2::dof] + 2/3*etac

    etacm = etac.reshape(mz,mx)
    zetacm = zetac.reshape(mz,mx)
    
    d2d3x = data['X_face_x']
    d2d3z = data['X_face_y']
    
    d2x = d2d3x[1::dof]
    d3x = d2d3x[2::dof]
    d2z = d2d3z[1::dof]
    d3z = d2d3z[2::dof]

    dd2x = d2x.reshape(mz,  mx+1)
    dd3x = d3x.reshape(mz,  mx+1)
    dd2z = d2z.reshape(mz+1,mx)
    dd3z = d3z.reshape(mz+1,mx)



    # Load data - hydrostatic pressure
    fout = fname+'_pstat'+ft  
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    ps = data['X_cell']
    pps = ps.reshape(mz,mx)

    #Load data - porosity
    fout = fname+'_phibk'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    solid_phase = data['X_cell']
    dof = 1
    poro = 1- solid_phase[0::dof]
    poro_m = poro.reshape(mz,mx)

    #Load data - total deviatoric stress
    fout = fname+'_stress'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    s1n = data['X_vertex']
    s1c = data['X_cell']

    # split into dofs
    dof = 4
    sxx1c = s1c[0::dof]
    szz1c = s1c[1::dof]
    sxz1c = s1c[2::dof]
    sII1c = s1c[3::dof]

    sxx1n = s1n[0::dof]
    szz1n = s1n[1::dof]
    sxz1n = s1n[2::dof]
    sII1n = s1n[3::dof]

    sII1cm = sII1c.reshape(mz,mx)

    #Load data - DP
    fout = fname+'_dp'+ft # DP stress
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    dp_data = data['X_cell']
    dof = 4
    dp = dp_data[0::dof]

    dpm = dp.reshape(mz,mx)


    #Load data - plastic strain rates
    fout = fname+'_yield'+ft
    imod = importlib.import_module(fout)
    data = imod._PETScBinaryLoad()
    ys_data = data['X_cell']
    dof = 4
    ys = ys_data[0::dof]

    ysm = ys.reshape(mz,mx)

    # Load data - strain rates
    fout = fname+'_strain'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    eps1n = data['X_vertex']
    eps1c = data['X_cell']

    # split into dofs
    dof = 4
    exx1c = eps1c[0::dof]
    ezz1c = eps1c[1::dof]
    exz1c = eps1c[2::dof]
    eII1c = eps1c[3::dof]

    exx1n = eps1n[0::dof]
    ezz1n = eps1n[1::dof]
    exz1n = eps1n[2::dof]
    eII1n = eps1n[3::dof]

    # get the deviatoric strain rates
    divc = exx1c + ezz1c
    exx1c = exx1c - 1.0/3.0*divc
    ezz1c = ezz1c - 1.0/3.0*divc
    eII1c = np.power(0.5*(np.power(exx1c,2)+np.power(ezz1c,2)+2.0*np.power(exz1c,2)) ,0.5)

    divn = exx1n + ezz1n
    exx1n = exx1n - 1.0/3.0*divn
    ezz1n = ezz1n - 1.0/3.0*divn
    eII1n = np.power(0.5*(np.power(exx1n,2)+np.power(ezz1n,2)+2.0*np.power(exz1n,2)) ,0.5)

    eII1cm = eII1c.reshape(mz,mx)
    divcm = divc.reshape(mz,mx)
    
    
    #velociy of the fluid phase: vf = vs - 1/phi(d2*grad(p) - d3)
    vfx = np.zeros([mz, mx+1])
    vfz = np.zeros([mz+1, mx])
    vfxc = np.zeros([mz, mx])
    vfzc = np.zeros([mz, mx])


    phimin = 1e-4
  
    for i in range(0,mx+1):
        for j in range(0,mz+1):

            if (j!=mz):
                if i ==0:
                    phi = poro_m[j][i]
                elif i== mx:
                    phi = poro_m[j][mx-1]
                else:
                    phi = 0.5*(poro_m[j][i-1]+poro_m[j][i])
                if phi < phimin:
                    phi = phimin
                vfx[j][i] = vxface[j][i] + 1/phi *(dd2x[j][i] * dpdx[j][i] + dd3x[j][i])

            if (i!=mx):
                if j ==0:
                    phi = poro_m[j][i]
                elif j == mz:
                    phi = poro_m[j-1][i]
                else:
                    phi = 0.5*(poro_m[j-1][i]+poro_m[j][i])

                if phi< phimin:
                    phi = phimin

                vfz[j][i] = vzface[j][i] + 1/phi *(dd2z[j][i] * dpdz[j][i] + dd3z[j][i])

    for i in range(0,mx):
        for j in range(0,mz):
            vfxc[j][i] = 0.5 * (vfx[j][i+1] + vfx[j][i])
            vfzc[j][i] = 0.5 * (vfz[j+1][i] + vfz[j][i])

    
    return mx, mz, xc, zc, xv, zv, fres0, vxc, vzc, pp, pps, poro_m, eII1cm, divcm, sII1cm, dpm, ysm

In [1]:
def load_fdpde_plasticstrain(fname, ft):

    import importlib
    import numpy as np

    
    # Load plastic strain
    fout = fname+'_evpbase'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    
    print(data.keys())
   # print(data['dof0'], data['dof1'], data['dof2'], data['dof3'])
    
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    
    evpbase = data['X_cell']
    kx = data['X_face_x']
    kz = data['X_face_y']
    
    # split into dofs
    dof = 4
    
    evptheta1 = evpbase[0::dof]
    evpmax1 = evpbase[1::dof]
    evpmin1 = evpbase[2::dof]

    evptheta = evptheta1.reshape(mz,mx)
    evpmax = evpmax1.reshape(mz,mx)
    evpmin = evpmin1.reshape(mz,mx)
    
    
    kxxx1 = kx[0::dof]
    kxzz1 = kx[1::dof]
    kxxz1 = kx[2::dof]
    kzxx1 = kz[0::dof]
    kzzz1 = kz[1::dof]
    kzxz1 = kz[2::dof]

    kxxx = kxxx1.reshape(mz, mx+1)
    kxzz = kxzz1.reshape(mz, mx+1)
    kxxz = kxxz1.reshape(mz, mx+1)
    kzxx = kzxx1.reshape(mz+1, mx)
    kzzz = kzzz1.reshape(mz+1, mx)
    kzxz = kzxz1.reshape(mz+1, mx)
    
    
    return evptheta, evpmax, evpmin, kxxx, kxzz, kxxz, kzxx, kzzz, kzxz


In [1]:
def load_fdpde_permeability(fname, ft):

    import importlib
    import numpy as np

    
    # Load plastic strain rates
    fout = fname+'_epsvp'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    
    devp = data['X_cell']
    
    # split into dofs
    dof = 4
    
    devpxx1 = devp[0::dof]
    devpzz1 = devp[1::dof]
    devpxz1 = devp[2::dof]

    devpxx = devpxx1.reshape(mz,mx)
    devpzz = devpzz1.reshape(mz,mx)
    devpxz = devpxz1.reshape(mz,mx)
    
    
    # Load plastic strain
    fout = fname+'_evp'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    
    evp = data['X_cell']
    
    # split into dofs
    dof = 4
    
    evpxx1 = evp[0::dof]
    evpzz1 = evp[1::dof]
    evpxz1 = evp[2::dof]

    evpxx = evpxx1.reshape(mz,mx)
    evpzz = evpzz1.reshape(mz,mx)
    evpxz = evpxz1.reshape(mz,mx)
    
    # Load plastic strain
    fout = fname+'_xkee'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    
    keex = data['X_face_x']
    keez = data['X_face_y']
    
    # split into dofs
    dof = 1
    
    keex1 = keex[0::dof]
    keez1 = keez[0::dof]

    keexm = keex1.reshape(mz,mx+1)
    keezm = keez1.reshape(mz+1,mx)

    
    return devpxx, devpzz, devpxz, evpxx, evpzz, evpxz, keexm, keezm


In [3]:
def load_coeff(nx,n,dt,itmax, ls, es, taus, c, ct, theta):

    etac = np.zeros([n,nx,itmax])
    zetac = np.zeros([n,nx,itmax])
    d2x = np.zeros([n,nx+1,itmax])
    d2z = np.zeros([n+1,nx,itmax])
    d3x = np.zeros([n,nx+1,itmax])
    d3z = np.zeros([n+1,nx,itmax])
    

    for i in range(itmax):

        t = i
        if (i<=9):
            ft = '_ts00'+str(t)
        elif (i<=99):
            ft = '_ts0'+str(t)
        else:
            ft = '_ts' + str(t)

        fname = fname0 + '_n'+str(n)
        
        etacm, zetacm, dd2x, dd3x, dd2z, dd3z = load_fdpde_coeff(fname, ft)

        etac[:,:,i] = etacm
        zetac[:,:,i] = zetacm
        d2x[:,:,i] = dd2x
        d2z[:,:,i] = dd2z
        d3x[:,:,i] = dd3x
        d3z[:,:,i] = dd3z
    return etac, zetac, d2x, d2z, d3x, d3z

In [ ]:
def load_stress_old(fname, ft):

    import importlib
    import numpy as np

    
    
    #Load data - total deviatoric stress
    fout = fname+'_stressold'+ft 
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()

    s1n = data['X_vertex']
    s1c = data['X_cell']

    # split into dofs
    dof = 4
    sxx1c = s1c[0::dof]
    szz1c = s1c[1::dof]
    sxz1c = s1c[2::dof]
    sII1c = s1c[3::dof]


    #Load data - DP
    fout = fname+'_dpold'+ft # DP stress
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    dp_data = data['X_cell']
    dof = 4
    dp = dp_data[0::dof]
 

    
    return sxx1c, szz1c, sxz1c, dp


In [1]:
def load_fdpde_coeff2(fname, ft):

    import importlib
    import numpy as np

    
    # Load data - coeff
    fout = fname+'_coefficient'+ft
    imod = importlib.import_module(fout) 
    data = imod._PETScBinaryLoad()
    
    mx = data['Nx'][0]
    mz = data['Ny'][0]
    
    etac_data = data['X_cell']
    etan = data['X_vertex']
    # split into dofs
    dof = 3
    etac = etac_data[1::dof]
    zetac = etac_data[2::dof] + 2/3*etac
    
    darcyC = etac_data[0::dof]

    etacm = etac.reshape(mz,mx)
    zetacm = zetac.reshape(mz,mx)
    
    darcyCm = darcyC.reshape(mz,mx)
    
    d2d3x = data['X_face_x']
    d2d3z = data['X_face_y']
    
    d2x = d2d3x[1::dof]
    d3x = d2d3x[2::dof]
    d2z = d2d3z[1::dof]
    d3z = d2d3z[2::dof]

    dd2x = d2x.reshape(mz,  mx+1)
    dd3x = d3x.reshape(mz,  mx+1)
    dd2z = d2z.reshape(mz+1,mx)
    dd3z = d3z.reshape(mz+1,mx)


    darcyC = etac_data[0::dof]
    
    return darcyCm
